In [1]:
%load_ext autoreload
%autoreload 2

import sys
import json
from tools import *
from pathlib import Path

/home/com/miniconda2/envs/py37/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
# https://dumps.wikimedia.org/backup-index.html

In [2]:
windows = [2, 5, 10]
min_count = 10
vector_sizes = [50, 100, 500]
languages = ['bxr', 'myv', 'kv']

In [12]:
# window = windows[1]
# vector_size = vector_sizes[1]
language = languages[1]

In [ ]:
# path = Path("./embeddings/"+str(vector_size)+"/"+str(window))
# wes = list(path.iterdir())
# for we in wes:
#     we.unlink()
# path.rmdir()
path.mkdir(parents=True)

In [3]:
wiki2text('/tmp/kvwiki-20211001-pages-articles.xml.bz2', 'kvwiki.txt')


number of articles: 2783


In [13]:
wiki2text(language+'wiki.xml.bz2', language+'.txt')

number of articles: 1623


In [9]:
for vector_size in vector_sizes:
    for window in windows:
        print("vector_size:", vector_size, "window:", window)
        pt = "./embeddings/"+language+"/"+str(vector_size)+"/"+str(window)+"/"
        w2v(vector_size, window, min_count, fname=pt+"cbow", fcorpus=language+".txt", sg=0)
        w2v(vector_size, window, min_count, fname=pt+"sg", fcorpus=language+".txt", sg=1)
        ft(vector_size, window, min_count, fname=pt+"ft", fcorpus=language+".txt")

vector_size: 50 window: 2
vector_size: 50 window: 5
vector_size: 50 window: 10
vector_size: 100 window: 2
vector_size: 100 window: 5
vector_size: 100 window: 10
vector_size: 500 window: 2
vector_size: 500 window: 5
vector_size: 500 window: 10


In [5]:
scores_d = {}
for language in ['bxr']:
    scores_d[language]={}
    for vector_size in vector_sizes:
        for window in windows:            
            pt = "./embeddings/"+language+"/"+str(vector_size)+"/"+str(window)+"/"
            path = Path(pt)
            wes = list(path.iterdir())
            print(wes)
            vocab = get_vocab(wes)
            print(len(vocab))
            for we in wes:
                l = we.stem
                
                if l not in scores_d[language]:
                    scores_d[language][l] = {}
                    
                if vector_size not in scores_d[language][l]:
                    scores_d[language][l][vector_size] = {}
                    
                print(we)
                scores = score("./evaluation/"+language+".json", we, vocab)
                scores_d[language][l][vector_size][window] = loss(scores).item()

[PosixPath('embeddings/bxr/50/2/ft'), PosixPath('embeddings/bxr/50/2/pmi'), PosixPath('embeddings/bxr/50/2/cbow'), PosixPath('embeddings/bxr/50/2/sg'), PosixPath('embeddings/bxr/50/2/glove')]
embeddings/bxr/50/2/ft 6124
embeddings/bxr/50/2/pmi 86330
embeddings/bxr/50/2/cbow 6124
embeddings/bxr/50/2/sg 6124
embeddings/bxr/50/2/glove 6125
6124
embeddings/bxr/50/2/ft
embeddings/bxr/50/2/pmi
embeddings/bxr/50/2/cbow
embeddings/bxr/50/2/sg
embeddings/bxr/50/2/glove
[PosixPath('embeddings/bxr/50/5/ft'), PosixPath('embeddings/bxr/50/5/pmi'), PosixPath('embeddings/bxr/50/5/cbow'), PosixPath('embeddings/bxr/50/5/sg'), PosixPath('embeddings/bxr/50/5/glove')]
embeddings/bxr/50/5/ft 6124
embeddings/bxr/50/5/pmi 86330
embeddings/bxr/50/5/cbow 6124
embeddings/bxr/50/5/sg 6124
embeddings/bxr/50/5/glove 6125
6124
embeddings/bxr/50/5/ft
embeddings/bxr/50/5/pmi
embeddings/bxr/50/5/cbow
embeddings/bxr/50/5/sg
embeddings/bxr/50/5/glove
[PosixPath('embeddings/bxr/50/10/ft'), PosixPath('embeddings/bxr/50/10

In [7]:
scores_d

{'bxr': {'ft': {50: {2: 0.2778688353647105,
    5: 0.3012650839698187,
    10: 0.31596595031395713},
   100: {2: 0.2802148802448385,
    5: 0.3025263207309292,
    10: 0.31668686171606214},
   500: {2: 0.27333772329384487,
    5: 0.29461021613638866,
    10: 0.31301374438054236}},
  'pmi': {50: {2: 0.1915103441963496,
    5: 0.18108895202798705,
    10: 0.17846559042252258},
   100: {2: 0.24404603684450865,
    5: 0.2282080275470952,
    10: 0.2269734618190567},
   500: {2: 0.38884923456708675, 5: 0.36220756326756526}},
  'cbow': {50: {2: 0.11707870393915454,
    5: 0.13447264845238993,
    10: 0.1573937893438196},
   100: {2: 0.1106555274258771,
    5: 0.12987812670761012,
    10: 0.15292283969683093},
   500: {2: 0.10156247854278538,
    5: 0.1246174197062411,
    10: 0.14977435630289163}},
  'sg': {50: {2: 0.1560990674898125,
    5: 0.1881720559397956,
    10: 0.2166377696261504},
   100: {2: 0.1542845541565819,
    5: 0.19299142821560064,
    10: 0.23264272779413478},
   500: {2: 0

In [23]:
import sys
for method, scores in scores_d['bxr'].items():
    print(method)
    for vector_size, windows in scores.items():
#         print(windows.keys())
        ws = [str(round(a,3)) for a in list(windows.values())]
        print("&",vector_size," & "," & ".join(ws),"\\\\","\\hline")

ft
& 50  &  0.278 & 0.301 & 0.316 \\ \hline
& 100  &  0.28 & 0.303 & 0.317 \\ \hline
& 500  &  0.273 & 0.295 & 0.313 \\ \hline
pmi
& 50  &  0.192 & 0.181 & 0.178 \\ \hline
& 100  &  0.244 & 0.228 & 0.227 \\ \hline
& 500  &  0.389 & 0.362 \\ \hline
cbow
& 50  &  0.117 & 0.134 & 0.157 \\ \hline
& 100  &  0.111 & 0.13 & 0.153 \\ \hline
& 500  &  0.102 & 0.125 & 0.15 \\ \hline
sg
& 50  &  0.156 & 0.188 & 0.217 \\ \hline
& 100  &  0.154 & 0.193 & 0.233 \\ \hline
& 500  &  0.147 & 0.189 & 0.228 \\ \hline
glove
& 50  &  0.422 & 0.381 & 0.37 \\ \hline
& 100  &  0.429 & 0.401 & 0.385 \\ \hline
& 500  &  0.433 & 0.405 & 0.395 \\ \hline


In [ ]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

language = 'bxr'
fname = language+".txt"

for vector_size in vector_sizes:
    for window in windows:
        pt = "./embeddings/"+language+"/"+str(vector_size)+"/"+str(window)+"/"
        wef = "/tmp/"+str(vector_size)+str(window)
        !glove/vocab_count -min-count $min_count -verbose 2 < $fname > /tmp/vocab.txt
        !glove/cooccur -memory 4.0 -vocab-file /tmp/vocab.txt -verbose 2 -window-size $window  < bxr.txt > /tmp/cooccurrence
        !glove/shuffle -memory 4.0 -verbose 2 < /tmp/cooccurrence > /tmp/cooccurrence.shuf
        !glove/glove -save-file $wef -threads 8 -input-file /tmp/cooccurrence.shuf -x-max 10 -iter 20 -vector-size $vector_size -binary 0 -vocab-file /tmp/vocab.txt -verbose 2

        glove_file = datapath(wef+".txt")
        tmp_file = datapath(pt+"glove")
        _ = glove2word2vec(glove_file,pt+"glove")

In [ ]:
for we in wes:
    print(we)
    scores = score("./evaluation/"+language+".json", we, vocab)
    print(loss(scores))

In [ ]:

ғаилә атай әсәй ағаһы һеңлеһе
семья папа мама брат сестра

илдә район ауыл ҡала
страна район село деревня

йылға волга  лена
река волга нил лена

теле урыҫ башҡорт татарскийй
язык русский бурятский

дин католик мосолман
религия католики мусульмане

тауҙар урал ҡаф
гора

фәндәре математика астрономия
науки

эсемлектәр һыу сәй һут һөт
напиток вода чай сок молоко простокваша

мал һарыҡ һыйыр аттар
скот овца корова конь

кейем кәпәс елән итек
одежда шапка рукавицы халат сапоги

яраҡ тәрилкә ҡалаҡ түңәрәге
посуда тарелка ложка уружка сковородка

ҡош төйлөгән ҡарлуғастар орел
птица коршун ласточка воробей орел
сустав
 
йөҙҙәре күҙҙәре танау ирен маңлайына
лицо глаза нос губа лоб

тәне аяғы  ҡулында арҡаһына күкрәгенә ҡорһаҡ
тело нога рука спина грудь живот

ҡояш
солнце роса туман град иней ливень

сәскә ҡыҙыл күк йәшел
цвет красный синий зеленый

туҡыма ебәк йөн бәрхәт
ткань шелк шерсть бархат

әҫәр йомаҡтар әйтемдәр әкиәт
книга загадка поговорка былина сказка

йорт ишек ҡапҡа диуар ҡыйыҡ баҫҡыстары
дом дверь ворота стена кровля лестница крыльцо

вкус
вкус сладкий соленый горький
 
мәктәп уҡытыусы уҡыу студенттар
школа учитель ученик студент

йыһан ғаләм йондоҙҙар лун
космос вселенная звезда луна

In [ ]:
%load_ext autoreload
%autoreload 2

windows = [2, 5, 10]
min_count = 10
vector_sizes = [50, 100, 500]
languages = ['bxr', 'myv', 'kv']

from pmi import *
language = languages[0]
print(language)

x = []
with open(language+".txt") as file:
    for l in file:
        x.append(l.strip())

for vector_size in vector_sizes:
    for window in windows:
        a = build_pairs(x, window)
        prepare_pmi(a)
        pmi(vector_size)
        convert("/tmp/space.dm", "./embeddings/"+language+"/"+str(vector_size)+"/"+str(window)+"/pmi")